In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('./data/train_10pct.csv')
df.head(5)

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.shape


(26059, 39)

In [20]:
df.mean().value_counts(normalize=True)

0.000153         0.064516
0.000000         0.032258
6261.106105      0.032258
0.007752         0.032258
0.076634         0.032258
0.042020         0.032258
2.126904         0.032258
0.066119         0.032258
0.762654         0.032258
0.000998         0.032258
0.000345         0.032258
526895.607237    0.032258
0.113128         0.032258
0.017844         0.032258
0.086074         0.032258
0.001228         0.032258
5.434898         0.032258
8.012817         0.032258
0.016808         0.032258
13.910434        0.032258
25.885682        0.032258
700.883457       0.032258
0.033271         0.032258
0.251698         0.032258
0.085422         0.032258
0.985571         0.032258
0.015081         0.032258
0.033961         0.032258
0.005027         0.032258
0.065275         0.032258
dtype: float64

In [4]:
df_labels = pd.read_csv('./data/train_10pct_labels.csv')
df_labels.head(5)

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [6]:
df_labels.shape


(26059, 2)

In [7]:
# read in data
train_values = pd.read_csv('./data/Proj5_train_values.csv')
train_labels = pd.read_csv('./data/Proj5_train_labels.csv')
# grab first 10% of rows
train_values = train_values.head(int(len(train_values) * 0.1))
train_labels = train_labels.head(int(len(train_labels) * 0.1))

In [9]:
print(f'Train value shape - {train_values.shape}')
print(f'Train labels shape - {train_labels.shape}')

Train value shape - (26060, 39)
Train labels shape - (26060, 2)


In [11]:
train_values.describe().T

,count,mean,std,min,25%,50%,75%,max
building_id,26060.0,526877.938219,303815.217966,17.0,261188.25,528759.0,791810.75,1052903.0
geo_level_1_id,26060.0,13.910130,8.007186,0.0,7.00,12.0,21.00,30.0
geo_level_2_id,26060.0,700.867575,412.529760,0.0,352.00,698.0,1050.00,1427.0
geo_level_3_id,26060.0,6260.985073,3669.304085,3.0,3040.75,6230.0,9462.00,12564.0
count_floors_pre_eq,26060.0,2.126938,0.730663,1.0,2.00,2.0,2.00,9.0
age,26060.0,25.884881,70.027047,0.0,10.00,15.0,30.00,995.0
area_percentage,26060.0,8.012893,4.413938,1.0,5.00,7.0,9.00,100.0
height_percentage,26060.0,5.434919,1.927906,2.0,4.00,5.0,6.00,32.0
has_superstructure_adobe_mud,26060.0,0.086071,0.280474,0.0,0.00,0.0,0.00,1.0
has_superstructure_mud_mortar_stone,26060.0,0.762663,0.425459,0.0,1.00,1.0,1.00,1.0


In [ ]:
normalize